# KoChatGPT 업그레이드 하기


***

In [1]:
import torch
import transformers

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

Torch version:1.12.1
Cuda version: 11.3
transformers version: 4.28.0
GPU 사용 가능여부: True


## 1. foundation model 변경하기

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/ko-gpt-trinity-1.2B-v0.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

우리가 사용할 모델의 토크나이저가 입력받아 처리할 수 있는 최대 토큰 수를 확인

In [3]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

gpt2 (small): 가장 작은 버전으로, 117M(1억 1천7백만)개의 파라미터를 가지고 있습니다. <br>
gpt2-medium: 345M(3억 4천5백만)개의 파라미터를 가진 중간 크기 모델입니다.<br>
gpt2-large: 774M(7억 7천4백만)개의 파라미터를 가진 큰 모델입니다.<br>
gpt2-xl: GPT-2 시리즈 중 가장 큰 모델로, 1.5B(15억)개의 파라미터를 가지고 있습니다.

토크나이징 방식 확인

In [4]:
import pandas as pd

input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
kogpt-2_tokens,▁바람,도,▁없는,▁공,중에,▁수직,의,▁파,문을,▁내,이며,▁고요,히,▁떨어지는,▁오,동,잎,은,▁누구,의,▁발자,취,▁입,니까.
Input_IDs,31140,20780,30359,30016,31373,41427,25792,30163,31047,30024,31111,51068,29936,36152,30027,20801,25846,25768,31199,25792,44202,27472,30148,37708


디코딩 방식 확인

In [5]:
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 



출력 결과에서 공백/빈줄이 나오는 이유? <br>
do_sample=False로 설정하면 모델이 가장 높은 확률의 토큰을 계속 선택하여 공백, 빈줄이 반복해서 나옴 -> **top-k 샘플링이나 top-p 샘플링**과 같은 **확률적 생성 방식**을 사용하여 다양한 토큰이 선택

#### top-k 샘플링 vs top-p 샘플링
![image.png](attachment:image.png)

빔 서치 디코딩을 사용하고 n-gram 패널티까지 부과

In [6]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.</d>


샘플링 기법 추가

Temperature 샘플링<br>
temperature는 확률 분포를 조절하여 텍스트의 다양성을 조정하는 매개변수입니다.<br>

temperature=1.0이면 원래 확률을 그대로 사용하지만, temperature<1일 때는 확률이 높은 단어가 더욱 강화되고, temperature>1일 때는 확률이 낮은 단어의 선택 가능성이 높아집니다.
<br>
예시: temperature=2.0으로 설정하면, 확률이 높은 단어 A의 확률이 상대적으로 낮아지고, 낮은 확률의 단어들도 선택 가능성이 높아집니다. 결과적으로 모델이 더 창의적인 선택을 하게 됩니다.


In [7]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 나는 그 발자국 소리에 귀기울입니다. 산새의 노랫소리도 귓전에 들립니다. 바람과 구름과 꽃의 속삭임에도 나는 가슴이 뜁니다. 내 생애 단 한번뿐인 사랑하는 이에게 줄 꽃 한 송이 꽃봉오리를 맺는 동안에도 내 눈에서는 오직 당신만 빛납니다. 당신만이 내 삶의 빛이고 행복입니다. 
 
 - 오정방 전남 강진에서 태어난 시인. 광주고와 전남대를 졸업했다. 1963년 조선일보 신춘문예에 당선돼 등단했다.


In [8]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.1)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 
 ------------------------------------------------ 
 오월의 마지막 날. 
 이 시를 읽으며 
 내 마음도 
 한없이 
 고요해지고 
 마음 한구석에 
 아련한 그리움이 피어오른다.</d>


### 오류 발생
원문이 그대로만 출력됨. 문제가 무엇일까?? <br>
num_beams도 7->10개로 트리처럼 여러 단어를 예측하게 수정해보았고, top_p=0.9->0.99까지 더 많은 단어를 포함시키게 하였다 -> 출력 변화 없음.<br>
<br>
오히려, top_p를 0.1로 낮추자 여러 단어가 출력되었다.

In [10]:
output_beam = model.generate(
    input_ids, 
    max_length=max_length, 
    num_beams=1, 
    no_repeat_ngram_size=2, 
    do_sample=True, 
    temperature=1.5, 
    top_k=80, 
    top_p=0.1
)
print(tokenizer.decode(output_beam[0]))


바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 
 
 <unk><unk>시집<unk>, <마음>, 문학과지성사, 1996, 시집.</d>


## 2. 데이터셋 확인

### 1) SFT 데이터셋

In [19]:
import json 
data_path_1_SFT = '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 import json                                                                                  │
│   2 data_path_1_SFT = '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'                  │
│ ❱ 3 with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:                          │
│   4 │   list_data_dict = json.load(json_file)                                                    │
│   5                                                                                              │
│   6 print(len(list_data_dict))                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '~/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'

**데이터는 질문과 답변 형식으로 존재**

### 2) RM 데이터셋

In [12]:
data_path_2_RM = './aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 data_path_2_RM = './aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'                    │
│ ❱ 2 with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:                           │
│   3 │   list_data_dict = json.load(json_file)                                                    │
│   4                                                                                              │
│   5 print(len(list_data_dict))                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: './aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'

**데이터는 질문과 답변 3개, 순위 형식으로 존재**

### 3) PPO 데이터셋

In [13]:
data_path_3_PPO = './aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 data_path_3_PPO = './aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'                  │
│ ❱ 2 with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:                          │
│   3 │   list_data_dict = json.load(json_file)                                                    │
│   4                                                                                              │
│   5 print(len(list_data_dict))                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: './aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'

**AI가 답할만한 적절한 질문으로 존재**

## 3. Supervised Fine-Tuning

라이브러리

In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

토크나이저

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저 교체
model = AutoModelForCausalLM.from_pretrained('skt/ko-gpt-trinity-1.2B-v0.5')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/ko-gpt-trinity-1.2B-v0.5', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

모델 인퍼런스 단계(모델이 학습 후 새로운 데이터에 대해 예측하는 과정) 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의

In [ ]:
from typing import Optional, Dict, Sequence #typing 모듈에서 옵션 타입, 딕셔너리 타입, 시퀀스 타입을 불러옵

class SFT_dataset(Dataset): #PyTorch의 Dataset을 상속받아 모델 학습에 필요한 데이터셋을 관리

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        #SFT_dataset 클래스의 생성자 메서드입니다. 데이터 경로와 토크나이저를 인자로 받으며, 데이터셋 로딩과 전처리를 수행
        super(SFT_dataset, self).__init__() #Dataset 클래스를 초기화
        logging.warning("Loading data...") #데이터셋 로딩 과정을 추적

        pattern_instruction = 'prompt'  # instruction -> 데이터셋에서 명령어를 나타내는 키를 지정
        pattern_output = 'completion'  # response -> 데이터셋에서 응답을 나타내는 키를 지정

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file: # 지정된 경로의 JSON 파일을 열고, 이를 파싱하여 리스트 형식의 데이터 딕셔너리로 부름
            list_data_dict = json.load(json_file) # list_data_dict에는 여러 개의 예시 데이터가 저장

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):" #명령어와 응답 형식을 지정한 템플릿 딕셔너리를 정의
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [ ]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
train_dataset = SFT_dataset(data_path_1_SFT='./aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

In [ ]:
# train_dataset.input_ids[0]를 디코딩해보세요.
def decode_tokens(dataset, tokenizer, index=0):
    # input_ids와 labels를 디코딩하여 원래 문장으로 변환
    input_text = tokenizer.decode(dataset.input_ids[index], skip_special_tokens=True)
    label_text = tokenizer.decode([token if token != -100 else tokenizer.pad_token_id for token in dataset.labels[index]], skip_special_tokens=True)
    
    return input_text, label_text

# 예시 사용법
input_text, label_text = decode_tokens(train_dataset, tokenizer)
print("Decoded Input:", input_text)
print("Decoded Label:", label_text)

In [ ]:
training_args = TrainingArguments(
    output_dir="/aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')